# Sentiment Analysis

In [1]:
from __future__ import unicode_literals
import re, codecs
from collections import Counter
from nltk.tokenize.api import TokenizerI
import sys
from os import path
import codecs
import itertools
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import re
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from hazm import *

## Dictionary-Based Approach

In [2]:
Posdic = ['مناسب','عالی','بهتر','برتر','شیک','زیبا','کارآزموده','گشاده','چابکی','ایستادگی','تازگی','آسوده','ستوده','شیفته','فراورده','استواری','شکوفیدن','کوشیدن','آمرزیده','برگزیده','گشوده','آراستن','آرِستن','بخشیده','غنوده','دلپذیر','ستودن','شایستن','نازیدن','بزرگ','دلیری','زیرکی','زبردستی','خشنودی','ورزیده','وارستگی','بسامانی','پیشرفته','مفید','تازه','ارزش','زیبندگی','آراسته','برازندگی','سريع','تندی','راستی','تميز','پاکیزگی','روشن','سبک','ارزان','ارزون','دلبسته','دلداده','دلشده','پاسداری','پرهیزکاری','خوشدلی','خوشی','رهایی','آسودن','رخشیدن','روان','قوی','آسون','پختگی','پایمردی','پایندانی','پایندگی','شيفته','زبانزد','پایداری','توانستن','بردباری','خجستگی','رستگاری','درخشندگی','خوبی','دانا','مهربان','روشنگر','\xa0خوب','بهبودی','پاکبازی','درخشیدن','خندان','پهلوانی','پیروزی','دلخوشی','بالیدن','بهروزی','خوشمزه']

Negdic = ['زشتی','بد','سرد','کثيف','آلوده','ربایندگی','پخمگی','خودکامگی','غلط','گران','گرون','چغلی','خرحمالی','دستمالی','دلخوری','تیغیدن','ریدن','درماندگی','رسوایی','دستگیری','تاخته','پلیدی','لاغر','يواشكی','خسته','تنگدستی','باختن','بازاندن','جنگیدن','شَخلیدن','غارتیدن','کِشتن','ضعيف','دردمندی','دریدگی','زنندگی','چاپیدن','ژولیده','آشفتن','\xa0سخت','خودشیرینی','دزدیدن','زاری','سوخته','سرافکنده','پایمال','پرافکنده','هراسیده','کوچک','کوتاه','كوچك','تنبل','آبكی','کوچيک','گذشته','واخوردگی','دریوزگی','دریدن','رهزنی','آزرده','فریفتن','سفت','پولكی','آسیب','تنگی','پوکیدن','پوکاندن','تاختن','تبیدن','ژولیدن','فرساییدن','مُردن','خودسری','نالیدن','آزدن','آزردن','سياه','آشفته','وامانده','افکنده','باخته','برشته','پایسته','تارانده','تفته','کشته','کشیده','خودکشی','سرشکسته','زدگی','تاریک','تلخ','ورشکسته','برانگیخته','پلاسیده','تنبیده','دریده','شکافته','غراشیده','گزیده','نازپرورده','تاریکا','تاریکی','وارسیدن','فروختن','درازدستی','وارفته','فرسودن','واخورده','پرحرف','ترکیده','وارفتگی','خارجی','وابسته','گریاندن','شکستن','گندیدن','وادادن','واریخته','درمانده','لهیده','نخراشیده','ورماندگی','بدهکاری','پژمردگی','چلچلی','گُسیختن','واسوختن','پژمرده','فریفته','نابسوده','وازده','تازانده','نارس','نارسیده','بازپرسی','بازجویی','بازرسی']


In [3]:
def clean(text):
    normalizer = Normalizer()
    # Remove URLs, User Mentions and Hashtags and Retweet and Number
    text = normalizer.normalize(re.sub(r"(?:\@|https?\://|\d+)\S+", "", text)
                                .replace("RT", "").replace("#", "").replace("_", " "))

    # Replace Repetitions of Punctuation and Charector
    text = ''.join(ch for ch, _ in itertools.groupby(text))

    # Remove Punctuation
    #text = re.sub(r'[^\w\s]', '', text)

    return text.strip()

In [4]:
stwd = ['همچنان', 'مدت', 'چیز', 'سایر', 'جا', 'طی', 'کل', 'کنونی', 'بیرون','های', 'مثلا', 'کامل','ها', 'کاملا','گیرد','شود','است', 'آنکه', 
            'موارد', 'واقعی', 'امور', 'اکنون', 'بطور', 'بخشی', 'تحت', 'چگونه', 'عدم', 'نوعی', 'حاضر', 'وضع', 'مقابل', 'کنار', 'خویش', 'نگاه', 'درون',
            'زمانی', 'بنابراین', 'تو', 'خیلی', 'بزرگ', 'خودش', 'جز', 'اینجا', 'مختلف', 'توسط', 'نوع', 'همچنین', 'آنجا', 'قبل', 'جناح', 'اینها', 'طور', 'شاید',
            'ایشان', 'جهت', 'طریق', 'مانند', 'پیدا', 'ممکن', 'کسانی', 'جای', 'کسی', 'غیر', 'بی', 'قابل', 'درباره', 'جدید', 'وقتی', 'اخیر', 'چرا', 'بیش',
            'روی', 'طرف', 'جریان', 'زیر', 'آنچه', 'البته', 'فقط', 'چیزی', 'چون', 'برابر', 'هنوز', 'بخش', 'زمینه', 'بین', 'بدون', 'استفاد', 'همان', 'نشان',
            'بسیاری', 'بعد', 'عمل', 'روز', 'اعلام', 'چند', 'آنان', 'بلکه', 'امروز', 'تمام', 'بیشتر', 'آیا', 'برخی', 'علیه', 'دیگری', 'ویژه', 'گذشته', 'انجام',
            'حتی', 'داده', 'راه', 'سوی', 'ولی', 'زمان', 'حال', 'تنها', 'بسیار', 'یعنی', 'عنوان', 'همین', 'هبچ', 'پیش', 'وی', 'یکی', 'اینکه', 'وجود'
            , 'شما', 'پس', 'چنین', 'میان', 'مورد', 'چه', 'اگر', 'همه', 'نه', 'دیگر', 'آنها', 'باید', 'هر', 'او', 'ما', 'من', 'تا', 'نیز', 'اما', 
            'یک', 'خود', 'بر', 'یا', 'هم','ای', 'را','دارد', 'این',"می", 'با','دارد','،',',','.', 'آن', 'برای', 'و', 'در', 'به', 'که', 'از']


In [15]:
from hazm import *

def Lexicon(Text):
    SentimentTotalDict = 0
    SentencesLevels = []
    Hassenses = []
    SentimentTotalSent = 0
    
    sentences = sent_tokenize(clean(Text)) 
    for sent in sentences:
        Hassense = 0
        totalsente = 0
        wordlist = word_tokenize(sent)
        for w in wordlist:
            if w in Posdic:
                SentimentTotalDict += 1
                totalsente += 1
                Hassense = 1
                print(w)
            elif w in Negdic:
                SentimentTotalDict -= 1
                totalsente -= 1
                Hassense = 1
                print(w)
        SentencesLevels.append(totalsente)
        Hassenses.append(Hassense)
                
    for x in SentencesLevels:
        if x > 0:
            SentimentTotalSent +=1
        elif x < 0:
            SentimentTotalSent -=1
    return(SentimentTotalDict,SentencesLevels,SentimentTotalSent,Hassenses)


In [10]:
Lexicon("زیبا بروفه آمد .علی پسر بسیار مهربان است :-). در مقابل رامین ژولیده و بد است! علی خوب نیست. فوتبال داریم.")


زیبا
مهربان
ژولیده
بد


(0, [1, 1, -2, 0, 0], 1, 1)

In [16]:
Lexicon("علی پسر خوبی و مهربان است. رامین بد است")


خوبی
مهربان
بد


(1, [2, -1], 0, [1, 1])

In [17]:
Lexicon("😃😃😃 علی پسر بسیار مهربان است. در مقابل رامین ژولیده و بد و نامرد است! علی خوبی؟ فوتبال هنر است.")

#(SentimentTotalDict,[Sentences Level],SentimentTotalSent,Hassense)

مهربان
ژولیده
بد
خوبی


(0, [1, -2, 1, 0], 1, [1, 1, 1, 0])

In [ ]:
"""
خیلی خیلی معمولیه ، موقع خرید درباره اسم سنهایزر تعریفشو زیاد شنیدم ولی بعد خرید و استفاده متوجه شدم محصوله معمولی هستش، البته اینو بگم قبلش از هندزفری urBeats 2 استفاده میکردم که در قیاس با سنهایزر مثل مقایسه بوگاتی با پرایده! از لحاظ طراحی خوبه ولی کیفیت صدا معمولی بیس پایین! به هیچ وجه قابل قیاس با هدفون های Beats نیست!
"""